# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298619514806                   -0.85    4.0
  2   -8.300222003161       -2.80       -1.25    1.5
  3   -8.300451868505       -3.64       -1.87    2.6
  4   -8.300463125059       -4.95       -2.70    2.0
  5   -8.300464453009       -5.88       -3.20    4.2
  6   -8.300464578394       -6.90       -3.32   19.5
  7   -8.300464605098       -7.57       -3.47    1.4
  8   -8.300464624670       -7.71       -3.61    1.1
  9   -8.300464639250       -7.84       -3.81    1.5
 10   -8.300464642435       -8.50       -3.96    1.0
 11   -8.300464643928       -8.83       -4.39    1.8


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64164917808                   -0.70    5.1
  2   -16.67841689862       -1.43       -1.14    1.6
  3   -16.67921665493       -3.10       -1.88    3.5
  4   -16.67927735305       -4.22       -2.75    4.8
  5   -16.67928569348       -5.08       -3.11    5.2
  6   -16.67928620383       -6.29       -3.53    1.2
  7   -16.67928621861       -7.83       -4.02    1.8


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32537219986                   -0.56    6.6
  2   -33.33332653607       -2.10       -0.99    1.0
  3   -33.33597202794       -2.58       -1.71    3.5
  4   -33.33681583006       -3.07       -2.25    3.4
  5   -33.33693424982       -3.93       -2.82    9.0
  6   -33.33694390368       -5.02       -3.86    5.0
  7   -33.33694391085       -8.14       -4.17    8.9


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298717386227                   -0.85    4.0
  2   -8.300298292698       -2.80       -1.59    1.0
  3   -8.300389706891       -4.04       -2.29   10.4
  4   -8.300314495219   +   -4.12       -2.17   13.9
  5   -8.300464292414       -3.82       -3.61    2.0
  6   -8.300464600514       -6.51       -3.96    5.2
  7   -8.300464640173       -7.40       -4.47    3.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.30366051038                   -0.56    6.6
  2   -33.31023603617       -2.18       -1.26    1.2
  3   -10.89487304178   +    1.35       -0.39    8.4
  4   -33.31088874319        1.35       -1.72    8.4
  5   -32.96139869714   +   -0.46       -1.17    3.9
  6   -31.95719184086   +    0.00       -0.99    4.5
  7   -33.32954767934        0.14       -1.97    5.8
  8   -33.33350235784       -2.40       -2.09    2.5
  9   -33.33588345861       -2.62       -2.34    2.5
 10   -33.33669953329       -3.09       -2.74    2.1
 11   -33.33685286276       -3.81       -3.00    2.8
 12   -33.33693424934       -4.09       -3.49    3.2
 13   -33.33694338037       -5.04       -3.84    3.5
 14   -33.33694357290       -6.72       -4.08    3.1


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.